In [1]:
from tqdm import tqdm
from analysis.nlp_transformations import separate_previous_conversation, remove_footers
from typing import List

from dao.email import DAOEmailGenerated, DAORealEmail
from dao.attribute import DAOAttribute

from models.attribute import AttributeInDB

from analysis.nlp_transformations import split_into_sentences, replace_links_with_text
from analysis.attribute_retriving import calc_distribution_sentence_length
import string

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading pl196x: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


In [2]:
text = "Oto kilkanaście zdań w języku polskim, które zawierają różnorodne formy interpunkcyjne i struktury zdaniowe. Jak się dzisiaj czujesz? Dlaczego niebo jest niebieskie? Czy wiesz, że Ks. Jan Twardowski był znanym polskim poetą? Uwaga! Proszę nie biec po korytarzu. Gdzie jest najbliższa apteka? Ile to kosztuje? To niesamowite, że możemy rozmawiać online! Co sądzisz o najnowszych odkryciach w kosmosie? Pamiętaj, że drzwi otwierają się o godz. 8:00 rano! Czy możesz mi podać sól? Kto zostanie nowym prezydentem Stanów Zjednoczonych? Wow! To było naprawdę ekscytujące. Jak możemy efektywniej oszczędzać energię? Czy znasz jakieś dobre przepisy na ciasto? Dr M. Nowak będzie dziś wygłaszać wykład na uniwersytecie. Czy myślisz, że sztuczna inteligencja może przewyższyć ludzką kreatywność?"
one_sentence = "I eat breakfast at 8:00 a.m."
split_into_sentences(text, 'pl')

['Oto kilkanaście zdań w języku polskim, które zawierają różnorodne formy interpunkcyjne i struktury zdaniowe.',
 'Jak się dzisiaj czujesz?',
 'Dlaczego niebo jest niebieskie?',
 'Czy wiesz, że Ks. Jan Twardowski był znanym polskim poetą?',
 'Uwaga!',
 'Proszę nie biec po korytarzu.',
 'Gdzie jest najbliższa apteka?',
 'Ile to kosztuje?',
 'To niesamowite, że możemy rozmawiać online!',
 'Co sądzisz o najnowszych odkryciach w kosmosie?',
 'Pamiętaj, że drzwi otwierają się o godz. 8:00 rano!',
 'Czy możesz mi podać sól?',
 'Kto zostanie nowym prezydentem Stanów Zjednoczonych?',
 'Wow!',
 'To było naprawdę ekscytujące.',
 'Jak możemy efektywniej oszczędzać energię?',
 'Czy znasz jakieś dobre przepisy na ciasto?',
 'Dr M. Nowak będzie dziś wygłaszać wykład na uniwersytecie.',
 'Czy myślisz, że sztuczna inteligencja może przewyższyć ludzką kreatywność?']

In [2]:
from numpy import std, var

dao_generated_emails: DAOEmailGenerated = DAOEmailGenerated()
dao_real_emails: DAORealEmail = DAORealEmail()
dao_attribute: DAOAttribute = DAOAttribute()


all_attributes: List[AttributeInDB] = dao_attribute.find_many_by_query({})

for attribute in tqdm(all_attributes, total=len(all_attributes), desc=f'Calculating sentences statistics', unit='emails', miniters=1):
    if attribute.referenced_db_name == 'email_generated_dataset':
        og_email = dao_generated_emails.find_one_by_query({'_id': attribute.referenced_doc_id})
        language = og_email.language
    else:
        og_email = dao_real_emails.find_one(attribute.referenced_db_name, {'_id': attribute.referenced_doc_id})
        language = og_email.detected_language
    _, og_text = separate_previous_conversation(og_email.text_plain)
    og_text = remove_footers(og_text)
    og_text = replace_links_with_text(og_text, "")
    
    words = [token for token in og_text.split() if token not in string.punctuation]
    # average_word_char_length = sum(len(word) for word in words) / len(words)
    char_data = [len(word) for word in words]
    # punctuation = len([char for char in og_text if char in ".,!?;:"])
    # 
    # split_sentences: List[str] = split_into_sentences(og_text, language)
    # number_of_sentences = len(split_sentences)
    # char_length_distribution, word_length_distribution = calc_distribution_sentence_length(split_sentences) 
    
    dao_attribute.update_one({'_id': attribute.id}, {'$set':{
        'standard_deviation_word_char_length': std(char_data),
        'variance_word_char_length':
         var(char_data)   
    }})


Calculating sentences statistics: 100%|██████████| 32724/32724 [01:08<00:00, 476.85emails/s]


In [6]:
dao_attribute: DAOAttribute = DAOAttribute()


all_attributes: List[AttributeInDB] = dao_attribute.find_many_by_query({})

for attribute in tqdm(all_attributes, total=len(all_attributes), desc=f'Calculating sentences statistics', unit='emails', miniters=1):
    punctuation_density = attribute.punctuation / attribute.number_of_characters
    punctuation_per_sentence = attribute.punctuation / attribute.number_of_sentences
    dao_attribute.update_one({'_id': attribute.id}, {'$set':{
        'punctuation_density': punctuation_density,
        'punctuation_per_sentence': punctuation_per_sentence,
    
    }})


Calculating sentences statistics: 100%|██████████| 32724/32724 [00:18<00:00, 1809.35emails/s]
